# Initialize

In [1]:
# Options

## Debugging level
debug_level = 0


In [2]:
# Initialize framework
constants, tools = init_processing(debug_level)
pn.extension("plotly")
hv.extension('bokeh', 'matplotlib')
%matplotlib inline


In [3]:
# Import all preprocessed data
invoice_data_by_customer            = pd.read_pickle(constants.PREPROCESSED_INVOICE_CUSTOMER_FILE_PATH)
invoice_data_by_service_start       = pd.read_pickle(constants.PREPROCESSED_INVOICE_SERVICEDATE_FILE_PATH)
invoice_data_by_invoice_date        = pd.read_pickle(constants.PREPROCESSED_INVOICE_INVOICEDATE_FILE_PATH)
forecast                            = pd.read_pickle(constants.PREPROCESSED_FORECAST_CLEANED_FILE_PATH)
revenue2020A                        = pd.read_pickle(constants.PREPROCESSED_REVENUE2020A_CLEANED_FILE_PATH)


In [ ]:
# Define functions

# Upgrade sequence
def fcn(df):
    # print(f"Size of df: {df.shape}")
    # dfs = df["Subscription"]
    # print(f"Contents of of df[Subscription]: {dfs}")
    # print(f"Contents of of df: {df}")

    customer_subscriptions_ordered_by_date = df.loc[:, ["Subscription", "Service Start"]].sort_values("Service Start")
    # subscriptions = df#.reset_index().loc[["Subscription", "Service Start"]]
    # subscriptions = df["Subscription"].values
    subscription_upgrade_sequence = customer_subscriptions_ordered_by_date["Subscription"].to_list()

    # print(customer_subscriptions_ordered_by_date)

    # TODO fcn to compute upgrade, downgrade, and delays between them.

    return subscription_upgrade_sequence


In [ ]:
invoice_data_by_customer.head(5)

In [ ]:
(atlas["Invoice Date"] - atlas["Service Start"]).apply(lambda date: date.days)

In [ ]:
# Plot delays between marketing initiation and payment date (NOTE: This is sample data and not accurate until relevant data is available.)
marketing_delay = atlas["Invoice Date"] - atlas["Service Start"]
marketing_delay_days = marketing_delay.apply(lambda date: date.days)
marketing_delay_df = pd.DataFrame(marketing_delay_days, columns=["Marketing Delay"])
# marketing_delay_df.plot.bar(figsize=(20, 10))
marketing_delay_df.topn(10) .plot.bar(figsize=(20, 10))

# TODO Group by customer regardless and Group by subscription length.
# TODO Response time for each state
# TODO Response time for each type of customer ("ATLAS Customer Type")
# TODO Response time for each type of customer ("Organization Record Type")
# TODO Data on potential customers and %success


In [ ]:
marketing_delay_df.plot.barh(figsize=(20, 30))

In [ ]:
# Basic info
total_invoiced_per_customer = atlas.groupby(["Customers"]).sum()
total_invoiced_per_customer.head()

In [ ]:
# Calculate conversion rates across various subscription combinations
repeat_customers = atlas.index.duplicated()
print(f"There are {repeat_customers.sum()} universities in this dataset who purchased 2 or more subscriptions.")
# atlas.loc[repeat_customers].groupby("Customers").apply(lambda x: x["Subscription"])
# atlas.loc[repeat_customers].groupby("Customers").apply(lambda x: x["Subscription"])

service_upgrades = pd.DataFrame( atlas.loc[repeat_customers].sort_values("Service Start").groupby("Customers").apply(fcn) )
service_upgrades.columns = [ "UpgradeSequence" ]
service_upgrades



# TODO Reduce #groups by also grouping by location, type of customer etc.

# TODO Add a category and grouping everywhere based on total or trailing invoice value (0-100k, 100k-1M, etc)

# TODO: Tag the following:
# - Drop-off in subscription length
# - Drop-off in subscription coverage
# - Upgrades with coverage gaps
# - Upgrades with no coverage gaps
# - Any coverage gaps

# TODO Find late payers past threshold

In [ ]:
# New customers
historical = atlas.reset_index().set_index("Service Start").sort_index().reset_index()
duplicated_customers = historical.reset_index()["Customers"].duplicated().apply(lambda x: not x)

new_customers = historical.loc[duplicated_customers].set_index("Service Start")
new_customers.drop(columns=['Customers', 'Number of Users', 'Invoice Date', 'Invoice #',
       'Invoice Amount', 'Subscription', 'Account Code ', 'Address', 'Lat',
       'Long', 'Service End', 'Subscription Duration'],
       inplace=True)
# new_customers["NumberOfNewUsers"] = pd.DataFrame( [np.ones([1, new_customers.shape[0]])] )
# new_customers

new_customers.hvplot.line(width=1000, height=400)


In [ ]:
np.ones([1, 10])
new_customers.shape

In [ ]:
# atlas.columns
atlas.loc[:, ["Subscription", "Service Start"]]
atlas.loc[:, ["Subscription", "Service Start"]].sort_values("Service Start")